## Contents
1. Importing libraries
2. Reading data, setting up
3. Grouping, aggregating, flagging data

### 1. Importing libraries

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os

### 2. Reading data, setting up

In [2]:
# Task 1.
# setting up filepath
path = r'C:\Users\Richárd\Desktop\CareerFoundry\Data Immersion\Archievment 4\Instacart_Basket_Analysis_2023_04_03\02_Data\Prepared_Data'
ords_prods_merge = pd.read_pickle(os.path.join(path, 'merged_flagged_data.pkl'))

### Grouping, aggregating, flagging data

In [3]:
# Task 2.
# Mean number of orders grouped by department_id
# plus some extra cleaning from 4.9.
ords_prods_merge.loc[ords_prods_merge['prices'] > 100, 'prices'] = np.nan
ords_prods_merge.groupby('department_id')['order_number'].mean()

department_id
1     15.457838
2     17.277920
3     17.170395
4     17.811403
5     15.215751
6     16.439806
7     17.225802
8     15.340650
9     15.895474
10    20.197148
11    16.170638
12    15.887671
13    16.583536
14    16.773669
15    16.165037
16    17.665606
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64

#### Task 3. Observations
- more departments are represented
- there is slightly more variance among the means
- department 10 and 21 stand out as having significantly more average order numbers than the rest
- there is no department that has significantly lower average amount of orders than the rest

In [4]:
# Task 4. Flagging customers based on loyality
# creating max_orders column based on grouping by user_id, aggregating max_order_numbers
ords_prods_merge['max_orders'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [5]:
# looping through max_orders column, adding flags to a list
# preferring looping to the loc method for readability and less function usage
loyalities = []
for x in ords_prods_merge['max_orders']:
    if x > 40:
        loyalities.append('Loyal customer')
    elif x <= 40 and x > 10:
        loyalities.append('Regular customer')
    else:
        loyalities.append('New customer')

# creating loyality_flag column, adding the list as rows
ords_prods_merge['loyality_flag'] = loyalities

# Checking output
ords_prods_merge['loyality_flag'].value_counts()

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyality_flag, dtype: int64

In [6]:
# if one wanted to do Task 4. with loc
# ords_prods_merge.loc[ords_prods_merge['max_orders'] > 40, 'loyality_flag'] = 'Loyal customer'
# ords_prods_merge.loc[(ords_prods_merge['max_orders'] > 10) & (ords_prods_merge['max_orders'] <= 40), 'loyality_flag'] = 'Regular customer'
# ords_prods_merge.loc[ords_prods_merge['max_orders'] <= 10, 'loyality_flag'] = 'New customer'

In [7]:
# Task 5. - average, min, max, price, grouped by loyality
ords_prods_merge.groupby(['loyality_flag']).agg({'prices' : ['min', 'max', 'mean', 'median']})
# There does not seem to be significant differences among the loyality groups


prices                       
                    min   max      mean median
loyality_flag                                 
Loyal customer      1.0  25.0  7.773575    7.4
New customer        1.0  25.0  7.801206    7.4
Regular customer    1.0  25.0  7.798262    7.4

In [8]:
# Task 6. Flagging users based on average spending
# creating avg_spending column
ords_prods_merge['avg_spending'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [9]:
# Looping through avg_spending, adding flags to a list
spending_flags = []
for spending in ords_prods_merge['avg_spending']:
    if spending >= 10:
        spending_flags.append('High spender')
    else:
        spending_flags.append('Low spender')

# Creating spending flags column from list
ords_prods_merge['spending_flag'] = spending_flags
# Checking if it worked
ords_prods_merge['spending_flag'].value_counts()

Low spender     32285131
High spender      119728
Name: spending_flag, dtype: int64

In [10]:
# Task 7. Flagging customers based on order frequencies
# creating median frequency column
ords_prods_merge['median_days_between_orders'] = ords_prods_merge.groupby(['user_id'])['days_after_previous_order'].transform(np.median)

In [11]:
# Looping through median_days_between_orders, adding flags to list
freq_flag = []
for days in ords_prods_merge['median_days_between_orders']:
    if days > 20:
        freq_flag.append('Non-frequent customer')
    elif days <= 20 and days > 10:
        freq_flag.append('Regular customer')
    else:
        freq_flag.append('Frequent customer')

# creating flag column
ords_prods_merge['order_frequency_flag'] = freq_flag

In [12]:
ords_prods_merge['order_frequency_flag'].value_counts()

Frequent customer        21559858
Regular customer          7208564
Non-frequent customer     3636437
Name: order_frequency_flag, dtype: int64

In [ ]:
ords_prods_merge.to_pickle(os.path.join(path, 'merged_data_with_groupings.pkl'))